# Azure AI Agents with Model Context Protocol (MCP) Support

本笔记本展示了如何使用 Azure AI Agents 和 Model Context Protocol (MCP) 工具创建一个智能代理，该代理可以利用外部 MCP 服务器来增强功能。


## 安装所需的 NuGet 包

首先，我们需要安装 Azure AI Agents Persistent 包，该包提供了使用 Azure AI Agents 的核心功能。


## 无密钥认证的优势

本笔记本展示了**无密钥认证**，它具有以下几个优势：
- ✅ **无需管理 API 密钥** - 使用基于 Azure 身份的认证
- ✅ **增强安全性** - 无需在代码或配置中存储密钥
- ✅ **自动凭据轮换** - Azure 负责凭据生命周期管理
- ✅ **基于角色的访问控制** - 使用 Azure RBAC 实现精细化权限管理

`DefaultAzureCredential` 会自动选择最佳的凭据来源：
1. 托管身份（在 Azure 中运行时）
2. Azure CLI 凭据（开发阶段）
3. Visual Studio 凭据
4. 环境变量（如果已配置）


In [1]:
#r "nuget: Azure.AI.Agents.Persistent, 1.1.0-beta.4"

Installed Packages Azure.AI.Agents.Persistent, 1.1.0-beta.4

安装 Azure Identity 包以使用 DefaultAzureCredential 进行 Azure 服务的身份验证。


In [2]:
#r "nuget: Azure.Identity, 1.14.2"

Installed Packages Azure.Identity, 1.14.2

## 导入所需命名空间

导入 Azure AI Agents 和 Azure Identity 所需的命名空间。


In [3]:
using Azure.AI.Agents.Persistent;
using Azure.Identity;

## 配置 Azure AI Agent 客户端（无密钥认证）

设置配置变量并通过**无密钥认证**创建 PersistentAgentsClient：
- **projectEndpoint**：Azure AI Foundry 项目的端点
- **modelDeploymentName**：已部署 AI 模型的名称（GPT-4.1 nano）
- **mcpServerUrl**：MCP 服务器的 URL（Microsoft Learn API）
- **mcpServerLabel**：用于标识 MCP 服务器的标签
- **DefaultAzureCredential**：使用托管身份、Azure CLI 或其他凭据来源（无需 API 密钥）


In [ ]:
var projectEndpoint = "Your Azure AI Foundry Project Endpoint";
var modelDeploymentName = "Your Azure OpenAI Model Deployment Name";
var mcpServerUrl = "https://learn.microsoft.com/api/mcp";
var mcpServerLabel = "mslearn";
PersistentAgentsClient agentClient = new(projectEndpoint, new DefaultAzureCredential());

## 创建 MCP 工具定义

创建一个 MCP 工具定义，用于连接到 Microsoft Learn MCP 服务器。这将使代理能够访问 Microsoft Learn 的内容和文档。


In [ ]:
MCPToolDefinition mcpTool = new(mcpServerLabel, mcpServerUrl);

## 创建 AI 代理

创建一个持久化的 AI 代理，使用指定的模型和 MCP 工具。该代理配置如下：
- 使用 GPT-4.1 nano 模型
- 指导其使用 MCP 工具进行辅助
- 访问 Microsoft Learn MCP 服务器


In [ ]:
PersistentAgent agent = await agentClient.Administration.CreateAgentAsync(
   model: modelDeploymentName,
   name: "my-learn-agent",
   instructions: "You are a helpful agent that can use MCP tools to assist users. Use the available MCP tools to answer questions and perform tasks.",
   tools: [mcpTool]
   );

## 创建线程并发送消息

创建一个对话线程，并发送一条用户消息，询问 Azure OpenAI 和 OpenAI 的区别。这将测试代理使用 MCP 工具提供准确信息的能力。


In [7]:
PersistentAgentThread thread = await agentClient.Threads.CreateThreadAsync();

// Create message to thread
PersistentThreadMessage message = await agentClient.Messages.CreateMessageAsync(
    thread.Id,
    MessageRole.User,
    "What's difference between Azure OpenAI and OpenAI?");

## 配置 MCP 工具资源（无密钥）

设置 MCP 工具资源。对于真正的无密钥方法，如果 MCP 服务器支持基于 Azure 身份验证，则可以移除自定义标头。以下示例展示了如何在需要时添加标头，但对于无密钥场景，这些可能并非必需。


In [ ]:
// Option 1: Completely keyless (if MCP server supports Azure identity)
MCPToolResource mcpToolResource = new(mcpServerLabel);
ToolResources toolResources = mcpToolResource.ToToolResources();

// Option 2: With custom headers (if still needed for specific MCP servers)
// MCPToolResource mcpToolResource = new(mcpServerLabel);
// mcpToolResource.UpdateHeader("Authorization", "Bearer <your-token>");
// ToolResources toolResources = mcpToolResource.ToToolResources();

## 开始代理运行

创建并启动一次运行以处理用户的消息。代理将使用配置的MCP工具和资源生成响应。


In [9]:
ThreadRun run = await agentClient.Runs.CreateRunAsync(thread, agent, toolResources);

## 监控运行并处理工具审批（无密钥）

监控代理运行状态并处理任何所需的工具审批。此循环：
1. 等待运行完成或需要操作
2. 在需要时自动批准 MCP 工具调用
3. 对于无密钥认证，如果 MCP 服务器支持 Azure 身份验证，则可能不需要标头


In [ ]:
while (run.Status == RunStatus.Queued || run.Status == RunStatus.InProgress || run.Status == RunStatus.RequiresAction)
{
    await Task.Delay(TimeSpan.FromMilliseconds(1000));
    run = await agentClient.Runs.GetRunAsync(thread.Id, run.Id);

    if (run.Status == RunStatus.RequiresAction && run.RequiredAction is SubmitToolApprovalAction toolApprovalAction)
    {
        var toolApprovals = new List<ToolApproval>();
        foreach (var toolCall in toolApprovalAction.SubmitToolApproval.ToolCalls)
        {
            if (toolCall is RequiredMcpToolCall mcpToolCall)
            {
                Console.WriteLine($"Approving MCP tool call: {mcpToolCall.Name}");
                
                // Option 1: Keyless approval (no headers needed)
                toolApprovals.Add(new ToolApproval(mcpToolCall.Id, approve: true));
                
                // Option 2: With headers (if required by specific MCP server)
                // toolApprovals.Add(new ToolApproval(mcpToolCall.Id, approve: true)
                // {
                //     Headers = { ["Authorization"] = "Bearer <your-token>" }
                // });
            }
        }

        if (toolApprovals.Count > 0)
        {
            run = await agentClient.Runs.SubmitToolOutputsToRunAsync(thread.Id, run.Id, toolApprovals: toolApprovals);
        }
    }
}

Approving MCP tool call: microsoft_docs_search


## 显示对话结果

检索并显示线程中的所有消息，包括用户的提问和客服的回复。消息按时间顺序排列，显示时间戳和角色标识。


In [12]:
using Azure;

AsyncPageable<PersistentThreadMessage> messages = agentClient.Messages.GetMessagesAsync(
    threadId: thread.Id,
    order: ListSortOrder.Ascending
);

await foreach (PersistentThreadMessage threadMessage in messages)
{
    Console.Write($"{threadMessage.CreatedAt:yyyy-MM-dd HH:mm:ss} - {threadMessage.Role,10}: ");
    foreach (MessageContent contentItem in threadMessage.ContentItems)
    {
        if (contentItem is MessageTextContent textItem)
        {
            Console.Write(textItem.Text);
        }
        else if (contentItem is MessageImageFileContent imageFileItem)
        {
            Console.Write($"<image from ID: {imageFileItem.FileId}>");
        }
        Console.WriteLine();
    }
}

2025-07-16 06:39:43 -       user: What's difference between Azure OpenAI and OpenAI?
2025-07-16 06:39:51 -  assistant: The main difference between Azure OpenAI and OpenAI lies in their deployment, management, and integration options:

1. **Azure OpenAI**:
   - A cloud service offered through Microsoft Azure.
   - Provides access to OpenAI models with additional enterprise features like security, compliance, and scale.
   - Allows integration with other Azure services, enabling seamless use within existing Azure-based solutions.
   - Offers managed deployment, monitoring, and support within the Azure ecosystem.
   - Suitable for organizations looking for enterprise-grade security, compliance, and regional availability.

2. **OpenAI (OpenAI API)**:
   - A standalone API service provided directly by OpenAI.
   - Accessible via the OpenAI platform without the need for Azure.
   - Focused on providing GPT models, DALL-E, etc., primarily for developers and researchers.
   - Suitable for indi


---

**免责声明**：  
本文档使用AI翻译服务[Co-op Translator](https://github.com/Azure/co-op-translator)进行翻译。尽管我们努力确保翻译的准确性，但请注意，自动翻译可能包含错误或不准确之处。原始语言的文档应被视为权威来源。对于关键信息，建议使用专业人工翻译。我们不对因使用此翻译而产生的任何误解或误读承担责任。
